## Obtain Text File

In [1]:
!wget -O "pg11.txt" "http://www.gutenberg.org/files/11/11-0.txt"

--2019-07-26 18:44:05--  http://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173595 (170K) [text/plain]
Saving to: ‘pg11.txt’

pg11.txt            100%[===================>] 169.53K  --.-KB/s    in 0.1s    

2019-07-26 18:44:11 (1.15 MB/s) - ‘pg11.txt’ saved [173595/173595]



## Remove License Data

In [0]:
!sed -i 1,38d pg11.txt
!head -n -367 pg11.txt > wonderland.txt

## Import Libraries

In [3]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


## Load Text File

In [0]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text_p = open(filename).read()
raw_text_p = raw_text_p.lower()

## Removing Punctuation

In [0]:
import re
raw_text = re.sub(r'[^\w\s]','',raw_text_p)

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

## Padding Sentence-Wise
#### Further Work necessary

In [7]:
from keras.preprocessing.sequence import pad_sequences
sentences_w = raw_text.split("\n")
sentences = []
for i in sentences_w:
  if len(i)> 0:
    sentences.append(i)
print(len(sentences))
sorted(list(set(' '.join(sentences))))
#find the length of each sentence
sentences_int = []
for sentence in sentences:
  char_int = [char_to_int[char] for char in sentence]
  sentences_int.append(char_int)
# define sequences
sequences = sentences_int
# pad sequence
padded = pad_sequences(sequences)
for i in padded[0:10]:
  print(len(i) , i[-1])
# prepare the dataset of input to output pairs encoded as integers
dataX = []
dataY = []
for sentence in padded:
  train_input = sentence[0: -1]
  train_output = sentence[-1] 
  dataX.append(train_input)
  dataY.append(train_output)
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
print(len(dataX),len(dataY) )
print (len(dataX[0]), dataX[0], dataY[0] )
print(len(dataX[1]), dataX[1], dataY[1])

2477
73 7
73 7
73 7
73 16
73 20
73 21
73 7
73 7
73 6
73 16
Total Patterns:  2477
2477 2477
72 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5 10  3 18
 22  7 20  1 11  1  6 17 25 16  1 22 10  7  1 20  3  4  4 11 22 10 17 14] 7
72 [ 0  0  0  0  3 14 11  5  7  1 25  3 21  1  4  7  9 11 16 16 11 16  9  1
 22 17  1  9  7 22  1 24  7 20 27  1 22 11 20  7  6  1 17  8  1 21 11 22
 22 11 16  9  1  4 27  1 10  7 20  1 21 11 21 22  7 20  1 17 16  1 22 10] 7


In [8]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  136014
Total Vocab:  29


In [9]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  135914


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## LSTM Model

In [11]:
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))

W0726 18:44:20.398203 140056985712512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [0]:
filepath="EIP_Session2_weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Training for 100 Epochs

In [13]:
# train the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=100, batch_size=512, callbacks = callbacks_list)

W0726 18:44:20.452980 140056985712512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 18:44:20.479672 140056985712512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 18:44:20.481420 140056985712512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 18:44:20.491974 140056985712512 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instruct

Epoch 1/100
135914/135914 [==============================] - 80s 587us/step - loss: 2.8844

Epoch 00001: loss improved from inf to 2.88441, saving model to EIP_Session2_weights-01-2.8844.hdf5
Epoch 2/100
135914/135914 [==============================] - 75s 551us/step - loss: 2.8068

Epoch 00002: loss improved from 2.88441 to 2.80681, saving model to EIP_Session2_weights-02-2.8068.hdf5
Epoch 3/100
135914/135914 [==============================] - 75s 549us/step - loss: 2.6987

Epoch 00003: loss improved from 2.80681 to 2.69868, saving model to EIP_Session2_weights-03-2.6987.hdf5
Epoch 4/100
135914/135914 [==============================] - 75s 552us/step - loss: 2.5796

Epoch 00004: loss improved from 2.69868 to 2.57957, saving model to EIP_Session2_weights-04-2.5796.hdf5
Epoch 5/100
135914/135914 [==============================] - 75s 555us/step - loss: 2.4606

Epoch 00005: loss improved from 2.57957 to 2.46058, saving model to EIP_Session2_weights-05-2.4606.hdf5
Epoch 6/100
135914/13591

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_1 (Dropout)          (None, 100, 1)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 29)                7453      
Total params: 796,957
Trainable params: 796,957
Non-trainable params: 0
_________________________________________________________________


In [15]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" ll

it was the white rabbit trotting slowly back again and looking
anxiously about as it went as if  "


## Predicting for 500 characters

In [29]:
# generate 500 characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	if(i%100 == 0):
		result = result + "\n"
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

 
suied anouw iere anice had been tooe anice tueeenlng tuiel suisting tunetiing butug tuastel taid ali
ce ierseled toaning tuieksis and teooied tuitting anice hellihtt taid the knew to tlated anice anice
 ruitedied suiersing tuastellng tuiaksed suizes oowtin gorumed suied anouw iere anice had been tooe 
anice tueeenlng tuiel suisting tunetiing butug tuastel taid alice ierseled toaning tuieksis and teoo
ied tuitting anice hellihtt taid the knew to tlated anice anice ruitedied suiersing tuastellng tuia
Done.
